# **Autoprice 데이터를 이용한 <span style="color:darkgreen">머신러닝</span> 문제**
---

> **<span style="color:red">다음 문항을 풀기 전에 </span>아래 코드를 실행하시오.**<br>
> 반드시 코드와 주석을 읽고 문제를 푸시오. <br>
> 반드시 출력된 데이터 설명을 읽고 문제를 푸시오.<br>
> [데이터 출처 & 설명](https://archive.ics.uci.edu/ml/datasets/automobile)도 읽고 문제를 푸시오.
---



중점적으로 복습하면 좋을 기능들 :

- [**]dataframe의 replace 사용방법

- [***]dataframe의 drop, dropna 사용법 (내가 원하는 row나 column을 제거하는 방법. 본인에게 익숙한 방법으로 프로그래밍하면 됩니다.)

- [***]dataframe의 새로운 column 생성하는 법.
  
  - df['new_columns'] = new_column_series

- [***]train_test_split 사용방법 (stratefy 또는 shuffle 옵션 참고 : [link](https://wikidocs.net/43332))

- [*]SimpleImputer 사용방법

- [***]get_dummies 사용방법 (first_drop=True 적용까지)





In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

cols = ['symboling', 'normalized_losses', 'make', 'fuel_type', 'aspiration', 'num_of_doors',
        'body_style', 'drive_wheels', 'engine_location', 'wheel_base', 'length', 'width', 'height',
        'curb_weight', 'engine_type', 'num_of_cylinders', 'engine_size', 'fuel_system', 'bore', 'stroke',
        'compression_ratio', 'horse_power', 'peak_rpm', 'city_mpg', 'highway_mpg', 'price']
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data', header=None, names=cols)

data.head(5)


,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horse_power,peak_rpm,city_mpg,highway_mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [ ]:
data.shape

(205, 26)

In [ ]:
len(data)

205

# Q1. data안의 모든 결측치는 '?'로 되어 있다. 모든 ?를 np.nan으로 바꾸시오.
-----------------------------

In [ ]:
####################
## your code here ##
####################

data.replace("?", np.nan, inplace = True)
data.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horse_power,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


# Q2. data의 컬럼들을 다음 조건에 맞추어 변환하시오.
-------------------------
* 대상 컬럼 : price, peak_rpm, horse_power, stroke, bore, normalized_losses 
* float64로 데이터 타입을 바꾼다.
--------------------------

In [ ]:
# 매우 가독성있는 방법
data = data.astype({'price':'float64', 
                    'peak_rpm':'float64', 
                    'horse_power':'float64', 
                    'stroke':'float64', 
                    'bore':'float64', 
                    'normalized_losses':'float64'}) 

In [ ]:
# apply 함수를 사용. 장점 : 함수만 변경하면, 데이터값을 변환하는 다양한 상황에서 호환성있게 사용할 수 있다. 

def to_float(element):
  return np.float64(element)

cols = ['normalized_losses', 'bore', 'stroke', 'horse_power', 'peak_rpm', 'price']

for col in cols:
  data[col] = data[col].apply(to_float)

In [ ]:
# solution

data[['normalized_losses', 'bore', 'stroke', 'horse_power', 'peak_rpm', 'price']] = data[['normalized_losses', 'bore', 'stroke', 'horse_power', 'peak_rpm', 'price']].astype(np.float64)

In [ ]:
data.head(15)

# Q3. data에서 price컬럼이 NaN인 row를 제거하고, 인덱스를 reset하시오.
----------------

In [ ]:
####################
## your code here ##
####################
'''
1. solution
data.drop(data[data['price'].isna()].index, inplace=True)
data.reset_index(drop=True, inplace=True)

2. dropna를 사용한 방식
'''
x = data['price'].dropna(axis=0)
data = data.loc[x.index]
data.reset_index(drop=True, inplace=True)

In [ ]:
len(data) #데이터 4개 소거

201

# Q4. data를 트레이닝셋과 테스트셋으로 분할하시오.
------------------
* price 컬럼이 y, 나머지 컬럼이 x가 된다.
* 변수명 규칙 :
    - x_train, x_test, y_train, y_test
* train : test = 8 : 2
* random state, seed 등은 2021로 고정
--------------------

In [ ]:
####################
## your code here ##
####################

from sklearn.model_selection import train_test_split

X = data.drop(['price'], axis=1)
Y = data['price']

In [ ]:
print("X shape : ", X.shape, "Y shape : ", Y.shape)

X shape :  (201, 25) Y shape :  (201,)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2021)

In [ ]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

((160, 25), (41, 25), (160,), (41,))

# Q5. data.info()를 확인하여 x안에 빈값이 있는 곳을 Imputing 하시오.
----------------------
* 모든 전처리 규칙은 트레이닝셋을 기준으로 찾는다.
    - x_train, x_test 전부 imputing한다.
* 데이터 타입이 object라면
    - 카테고리 데이터 취급을 한다.
    - 최빈값으로 imputing 한다.
* 데이터 타입이 object가 아니라면
    - 연속형 데이터 취급을 한다.
    - 평균값으로 imputing 한다.
-----------------------

In [ ]:
data.info()

In [ ]:
####################
## your code here ##
####################

from sklearn.impute import SimpleImputer

imputer_num = SimpleImputer(strategy='mean') # 평균 imputing 객체생성
imputer_ca = SimpleImputer(strategy='most_frequent') # 최빈값 imputing 객체생성

num_cols = ['normalized_losses', 'bore', 'stroke', 'horse_power', 'peak_rpm'] # NaN이 있었던 numeric columns
ca_cols = ['num_of_doors'] # NaN이 있었던 categorical columns

x_train[ca_cols] = imputer_ca.fit_transform(x_train[ca_cols]) #imputing을 한 뒤, 기존 column에 assign
x_train[num_cols] = imputer_num.fit_transform(x_train[num_cols]) #imputing을 한 뒤, 기존 column에 assign

x_test[ca_cols] = imputer_ca.transform(x_test[ca_cols]) #imputing을 한 뒤, 기존 column에 assign
x_test[num_cols] = imputer_num.transform(x_test[num_cols]) #imputing을 한 뒤, 기존 column에 assign

# Q6. 모든 x안의 카테고리 데이터를 더미변수들로 바꾸시오.
---------------------
* 모든 전처리 규칙은 트레닝셋을 기준으로 찾는다.
* x_train.info()를 확인하여 object라면 카테고리 데이터 취급한다.
    - 단, 'symboling'변수도 더미변수로 만든다.
* 카테고리 데이터 안에 3개 클래스가 있다면, 2개 컬럼이 만들어져야 한다.
* 원본 컬럼은 제거한다.
* 더미 변수 제작 후, x_test의 컬럼들은 x_train의 컬럼과 같아야 한다.
    - x_train에 없는 x_test의 컬럼은 제거를 해야 한다.
    - x_train에만 있는 컬럼은 x_test에도 만들어야 한다.(0으로 채운다.)
------------------------------

In [ ]:
####################
## your code here ##
####################

x_train = pd.get_dummies(x_train, columns = ['symboling', 'make', 'fuel_type', 'aspiration',
       'num_of_doors', 'body_style', 'drive_wheels', 'engine_location',
       'engine_type', 'num_of_cylinders', 'fuel_system'], drop_first=True)

x_test = pd.get_dummies(x_test, columns = ['symboling', 'make', 'fuel_type', 'aspiration',
       'num_of_doors', 'body_style', 'drive_wheels', 'engine_location',
       'engine_type', 'num_of_cylinders', 'fuel_system'], drop_first=True)

temp = pd.DataFrame()

for col in x_train.columns :
    try :
        temp[col] = x_test[col]
    except :
        temp[col] = 0
x_test = temp

In [ ]:
print("train shape : ", x_train.shape)
print("test shape : ", x_test.shape)

train shape :  (160, 66)
test shape :  (41, 66)


# Q7. 선형회귀 모델을 학습시키시오.
---------------------
* 모델을 lr에 선언하시오.
---------------------

In [ ]:
####################
## your code here ##
####################
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
print("weight의 수 :", lr.coef_.shape, "bias : ", lr.intercept_)

weight의 수 : (66,) bias :  -6919.302088719043


# Q8. 트레이닝셋에서, 'aspiration_turbo'가 1인 경우만을 이용하여 선형회귀 모델을 학습시키시오.
----------------
* 모델을 lr_turbo 에 선언하시오.
* 전처리 방식에 따라서 aspiration_std가 0인 경우를 골라야할 수도 있다.
--------------

In [ ]:
####################
## your code here ##
####################
from sklearn.linear_model import LinearRegression

lr_turbo = LinearRegression()

x_train_t = x_train[x_train['aspiration_turbo']==1]
y_train_t = y_train[x_train['aspiration_turbo']==1]

lr_turbo.fit(x_train_t, y_train_t)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Q9. 트레이닝셋에서, 'aspiration_turbo'가 0인 경우만을 이용하여 선형회귀 모델을 학습시키시오.
----------------
* 모델을 lr_std 에 선언하시오.
* 전처리 방식에 따라서 aspiration_std가 1인 경우를 골라야할 수도 있다.
--------------

In [ ]:
####################
## your code here ##
####################
from sklearn.linear_model import LinearRegression

lr_std = LinearRegression()

x_train_s = x_train[x_train['aspiration_turbo']==0]
y_train_s = y_train[x_train['aspiration_turbo']==0]

lr_std.fit(x_train_s, y_train_s)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Q10. lr, lr_turbo, lr_std 를 테스트셋 위에서 평가하시오.
--------------------
* RMSE를 출력한다.
* lr_turbo는 테스트셋에서, aspiration_turbo가 1인 경우만을 이용하여 평가한다.
* lr_std는 테스트셋에서, aspiration_turbo가 0인 경우만을 이용하여 평가한다.
-----------------------------------------------

In [ ]:
####################
## your code here ##
####################
from sklearn.metrics import mean_squared_error as MSE

x_test_t = x_test[x_test['aspiration_turbo']==1]
y_test_t = y_test[x_test['aspiration_turbo']==1]

x_test_s = x_test[x_test['aspiration_turbo']==0]
y_test_s = y_test[x_test['aspiration_turbo']==0]

y_pred = lr.predict(x_test)
y_pred_t = lr_turbo.predict(x_test_t)
y_pred_s = lr_std.predict(x_test_s)

rmse = MSE(y_test, y_pred)**.5
rmse_turbo = MSE(y_test_t, y_pred_t)**.5
rmse_std = MSE(y_test_s, y_pred_s)**.5

print(f"전체 모델의 RMSE : {rmse:.4f}")
print(f"터보 모델의 RMSE : {rmse_turbo:.4f}")
print(f"std 모델의 RMSE : {rmse_std:.4f}")


전체 모델의 RMSE : 2047.0041
터보 모델의 RMSE : 72706.7265
std 모델의 RMSE : 2273.9895
